In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from  keras.models import Sequential, Model, clone_model
from  keras.layers import Input, Dense, Add,LeakyReLU, Activation, Concatenate,Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,Callback
#from  keras.applications.vgg16 import VGG16, preprocess_input
#from  keras.applications import ResNet152V2,ResNet101V2,ResNet50V2 
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input 

from  keras.optimizers import RMSprop, Adam, Adadelta
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
import keras.regularizers as regularizers
from tensorflow.keras.backend import epsilon
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw,ImageFont

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras.backend as K
import numpy as np
import random

In [4]:
BATCH_SIZE = 32
IMG_SIZE = 224

# Generate models architecture

## Bounding Boxer

In [5]:
model2 =  MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3), alpha = 1.4)
input_ = model2.input

In [6]:
boxer_branch = model2.layers[-1].output
boxer_branch = Conv2D(4,7, strides=(1,1), name="detector_conv_2", activation = "relu")(boxer_branch)
boxer_branch = Reshape((4,), name = "coords")(boxer_branch)
#model_boxer = Model(input = [input_], outputs = boxer_branch)
#model_boxer = multi_gpu_model(model_boxer,gpus=2)
#model_boxer.load_weights('mobile_net_seconda_passata.h5')

## Classifier

In [ ]:
model =  MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3), alpha = 1.4)
model.trainable = True
w = 0
for layer in model.layers[:len(model.layers)-w]:
    layer.trainable = True
    layer.name = layer.name + "_class"
last = model.layers[-1].output
classifier_branch = GlobalAveragePooling2D(name = "glob_average_mine")(last)
#classifier_branch = Dense(128, activation = "relu", name = "dense1")(classifier_branch)
classifier_branch = Dense(1, activation = "sigmoid", name = "classes")(classifier_branch)
#model_class = Model(inputs=[model.input], outputs=[classifier_branch])
#model_class = multi_gpu_model(model_class,gpus=2)
#model_class.load_weights("mobile_net_class.h5")

## Merge models

In [8]:
parallel_model= Model([model.input, model2.input], outputs=[classifier_branch, boxer_branch])

In [9]:
parallel_model.load_weights("single_face_total.h5")
#parallel_model.save_weights("single_face_total.h5")

In [10]:
parallel_model.compile(optimizer = "adam", loss="mse")

# Testing

In [16]:
main_folder_path = "/home/stach/Desktop/Machine_learning_project/dataset/"
test_df = pandas.read_csv(main_folder_path+"FACE_CLASSIFIER/test_finale.csv")
test_df.head(4)

,image_path,face
0,/home/stach/Desktop/Machine_learning_project/d...,1
1,/home/stach/Desktop/Machine_learning_project/d...,1
2,/home/stach/Desktop/Machine_learning_project/d...,1
3,/home/stach/Desktop/Machine_learning_project/d...,0


test_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/dataset_face_classifier/test_borderline.csv")
test_df.head(4)

In [17]:
test_datagen = ImageDataGenerator(#samplewise_center = True,
    #samplewise_std_normalization = True,
    preprocessing_function = preprocess_input
)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=None,
        x_col="image_path",
        y_col=["face"],
        class_mode="raw",
        shuffle=False,
        color_mode = 'rgb',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)        
STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 

def generate_data_generator2(generator):
    while True:
        batch_1 = generator.next()
        yield [batch_1[0],batch_1[0]] 

#print(test_generator[0][0][0].shape)
#image_yes = test_generator[200][0]
#image_no = test_generator[20][0]
#pred1=parallel_model.predict([image_yes,image_yes])
#pred2=parallel_model.predict([image_no,image_no])
pred=parallel_model.predict_generator(generate_data_generator2(test_generator), steps = STEP_SIZE_TEST,verbose=1)

Found 8579 validated image filenames.
269/268 [==============================] - 762s 3s/step


In [18]:
print(pred)

[array([[0.99999976],
       [0.9999999 ],
       [0.96014893],
       ...,
       [1.        ],
       [0.99999595],
       [1.        ]], dtype=float32), array([[ 44.859447,  58.38406 , 118.70114 , 127.98302 ],
       [ 45.841595,  20.133825,  70.085884, 147.12196 ],
       [ 23.873426,   0.      , 171.58652 , 209.17447 ],
       ...,
       [112.348434,  37.31625 ,  30.018988,  57.31039 ],
       [ 80.308624,  25.475565,  53.96    ,  97.43355 ],
       [ 52.552242,  30.05226 , 119.46674 , 108.74912 ]], dtype=float32)]


In [20]:
import os
from PIL import ImageFont
font = ImageFont.truetype("usr/share/fonts/liberation/LiberationSans-Bold.ttf", 40)
def plot_results(image_batch, pred_class, pred_box):
    count = 0 
    for el in zip(image_batch,pred_class,pred_box): 
        count = count +1 
        if count%100==0:
            print(count)
        dir_ = os.path.dirname(el[0]).split("/")[-2]
        base = os.path.basename(el[0])
        base = os.path.splitext(base)[0]
        #im = Image.fromarray(np.uint8(el[0]*255))
        im = Image.open(el[0])
        im = im.resize((224,244))
        img1 = ImageDraw.Draw(im)
        if el[1][0]>=0.5:
            bb = el[2]
            img1.rectangle([(bb[0],bb[1]),(bb[0]+bb[2],bb[1]+bb[3])], outline ="red") 
        #img1.text(xy=(0,0), text = str(round(el[1][0],2)),fill=(255,255,255),font = font)
        img1.text(xy=(0,0),text = str(round(el[1][0],2)), fill=((255)), font=font)
        im.save(main_folder_path + "results/"+ dir_+"_"+base + "_out.png")
        #if count == 5000:
        #    break

In [21]:
plot_results(test_generator.filenames,pred[0], pred[1])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
